In [ ]:
%run ../commons/imports_legal.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
create_database('blp_dataset')
create_table('products_tmp_amazon')
cursor = conn.cursor()

# == Files config ==
MAIN_FOLDER = r"D:\legal_data\amazon-pqa\d_amazon_pqa_accessories.json"

In [ ]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()
    
    return clear_content

In [ ]:
folder = Path(f"{MAIN_FOLDER}")

# for file in  os.listdir(folder):
#     file_path = Path(f"{MAIN_FOLDER}\{file}")
#     print(file_path)
    
for line in open(folder, 'r'):
    data = json.loads(line)

    description = clear_text(data['product_description'])
    name = clear_text(data['item_name'])

    query = "INSERT INTO products_tmp_amazon (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
    values = (name, description, "Amazon (PQA)", "f")

    cursor.execute(query, values)
    conn.commit()